In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import os
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np 
import glob
import time
import yfinance as yf
import datetime


# options = Options()
# options.add_argument("--start-maximized")  # Maximize the browser window
# options.add_argument("--disable-extensions")
# options.add_argument("--disable-popup-blocking")
# options.add_argument("--disable-infobars")
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")
# options.add_argument("--disable-gpu")
# options.add_argument("--ignore-certificate-errors")

#driver = webdriver.Chrome()
#driver = webdriver.Safari()

ERROR = False

cookies_ec_dict = {'google': EC.element_to_be_clickable((By.XPATH, "//div[@class='QS5gu sy4vM']")),
                'nasdaq': EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'investing.com':EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'zacks.com_readmore': EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'Button__StyledButton') and contains(text(), 'Read more to accept preferences')]")),
                'zacks.com': EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'Button__StyledButton-a1qza5-0') and contains(text(), 'Accept all')]"))}

nav_nasdaq_ec_dict = {'select_date': EC.element_to_be_clickable((By.ID, "date-picker__toggle")),
                            #"//button[@class='date-picker__toggle' and @aria-label='click show datepicker popup']")),      
                      'select_searchbox':  EC.visibility_of_element_located((By.CLASS_NAME, "date-picker__input")),
                      'apply_date_button': EC.element_to_be_clickable((By.XPATH,
                            "//button[@class='date-picker__apply']"))}

nav_investingcom_ec_dict = {'select_calendar': EC.element_to_be_clickable((By.ID, "datePickerToggleBtn"))}

def accept_cookies(site = 'google'):   
    if site == 'zacks.com':
        try:
            read_more = WebDriverWait(driver, 1).until(cookies_ec_dict['zacks.com_readmore']) 
            read_more.click()
        except:
            print("No 'Read more' button found on ", site)
    try:
        accept = WebDriverWait(driver, 1).until(cookies_ec_dict[site])
        accept.click()
    except:
        print("No 'Accept' button found or unable to click on ", site)


def google(query, cookies):
    # Open the website
    driver.get("https://google.com")
    if cookies: accept_cookies('google')
    search_box = driver.find_element(By.NAME, "q")

     # Enter your search query
    search_query = query
    search_box.send_keys(search_query)

    # Simulate hitting Enter
    search_box.send_keys(Keys.RETURN)

def click_google_site(site):
    driver.find_element(By.XPATH, f"//a[contains(@href, '{site}')]").click()

def select_date(year, month, day, site = 'nasdaq'):
    if site == 'nasdaq':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['select_date']).click()
        driver.execute_script(f"document.querySelector('.date-picker__input').value = '{date}';")
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['e_button']).click()
    elif site == 'investing.com':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_investingcom_ec_dict['select_calendar']).click()
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#startDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#endDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        driver.find_element(By.ID, 'applyBtn').click()
            
def google_site(query, site, cookies):
    google(query, cookies)
    click_google_site(site)


def get_symbols(site, openclose = False):
    time.sleep(1)

    symbols = []
    if openclose: timings = []
    if site == 'investing.com':
        openclose_dict = {'1': 'open', '2': 'none', '3': 'close'}
        table = driver.find_element(By.ID, "earningsCalendarData")
        tbody = table.find_element(By.TAG_NAME, "tbody")
        rows = tbody.find_elements(By.TAG_NAME, "tr")
        # Iterate over each table row
        for index, row in enumerate(rows):
            if index == 0: continue
            try:
                inner_html = row.get_attribute("innerHTML")
                soup = BeautifulSoup(inner_html, "html.parser")
                a_element = soup.find("a", class_="bold middle")
                if openclose:
                    timing_element = soup.find("td", class_="right time")
                    timing_value = timing_element.get("data-value")
                    timings.append(openclose_dict[timing_value])
                if a_element:
                    company = a_element.text.strip()
                    symbols.append(company)
            except Exception as e:
                print(f"Error processing row {index}: {e}")
                continue
    if openclose: return symbols, timings
    return symbols

def search_on(website, query):
    if website == 'zacks.com':
        search_input = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.ID, "search-q"))
        )
        # Input the search query
        search_input.send_keys(query)
        # Press ENTER to submit the form
        search_input.send_keys(Keys.ENTER)


def get_rank(website):
    if website == 'zacks.com':
        try:
            # Wait for the paragraph element to be visible
            rank_paragraph = WebDriverWait(driver, 20).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "rank_view"))
            )

            # Get the text of the paragraph element
            rank_text = rank_paragraph.text.strip()
            return int(rank_text[0])
        
        except Exception as e:
            ERROR = True

            
def analyst(website, symbol, cookies):
    if website == 'zacks.com':
        driver.get(f'https://www.zacks.com/stock/quote/{symbol}?q={symbol}')
    else:
        google_site(f'{website[:website.find(".")]} {symbol}', website, cookies)
    if cookies: accept_cookies(website)
    rank = get_rank(website)
    return rank
   
def earnings_rank(website, year, month, day, path = ''):
    if path != '':
        if path[-1] == '/': path = path[:-1]
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:   
        print('path does not exist')
    else:
        for index, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
            ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
            rank = analyst(website, ticker, cookies = not(index))
            print(ticker, rank)
            with open(tickerpath+website, 'a') as f:
                f.write(f'{datetime.datetime.today().year}-{datetime.datetime.today().month}-{datetime.datetime.today().day},    {rank}\n')


def get_tickers_for_earnings_date(year, month, day, openclose = False):
    google_site('earnings calendar', 'investing.com', cookies = True)
    accept_cookies('investing.com')
    time.sleep(0.5)
    select_date(year, month, day, site = 'investing.com')
    if openclose: 
        tickers, timings = get_symbols('investing.com', openclose = openclose)
        return tickers, timings
    else:
        tickers = get_symbols('investing.com', openclose = openclose)
        return tickers
 

def make_earnings_dates_directories(path, year, month, day, tickers, openclose  = None):
    if path[-1] == '/': path = path[:-1]
    if os.path.exists(path) is False:
        os.mkdir(path)
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:
        os.mkdir(path+'/'+f'{year}-{month}-{day}')  
    for index, ticker in enumerate(tickers):
        if os.path.exists(path+'/'+f'{year}-{month}-{day}/'+ ticker) is False:
            os.mkdir(path+'/'+f'{year}-{month}-{day}/'+ ticker)
        if openclose is not None:
            with open(path+'/'+f'{year}-{month}-{day}/'+ ticker + '/openclose.dat', 'a') as f:
                f.write(openclose[index])

    
         
def filter_by_lastrank(path, year, month, day, website):
    if path != '':
        if path[-1] == '/': path = path[:-1]
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:   
        print('path does not exist')
    else:
        ranks = ['1','2','3','4','5','None']
        ticker_rank = {rank:[] for rank in ranks}
        for _, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
            ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
            with open(tickerpath+website, 'r') as f:
                text = f.read()
                last = text.split(' ')[-1]
                ticker_rank[last[:last.find('\n')]].append(ticker)

        return ticker_rank



##########################################


In [102]:
driver = webdriver.Chrome()
year, month, day = 2024, 3, 19
tickers, timings = get_tickers_for_earnings_date(year, month, day, openclose = True)
driver.quit()

In [103]:
for tic, tim in zip(tickers, timings):
    print(tic, tim)

ZTO none
TME open
CNM none
XPEV open
AIR none
SLNO none
EVV none
ORLA none
ORIC none
HUYA open
ABSI none
DEC open
ACIU none
SCPH none
CLRB none
KGEI none
CMCM none
KRON none
DRIO none
AATC none
SNCE none
XGN none
SOLO none
UAMY none
SSKN none
TBIO none
NROM none
JT none
WEBB none
GADS none
GENN none
GCFB none


In [104]:
make_earnings_dates_directories('earnings', year, month, day, tickers, openclose = timings)


In [105]:
driver = webdriver.Chrome()
earnings_rank('zacks.com', year, month, day, path = 'earnings')
driver.quit()

No 'Read more' button found on  zacks.com
GCFB None
KGEI 5
XPEV 3
CNM 3
UAMY None
SLNO 4
JT None
SCPH 4
SNCE None
AATC None
CLRB 3
KRON 4
ORIC 3
ORLA 4
AIR 4
WEBB None
DRIO 4
HUYA 3
SOLO 3
CMCM None
EVV None
DEC 4
TBIO 3
ZTO 4
ABSI 2
NROM None
ACIU 3
SSKN 3
GENN None
GADS None
XGN 2
TME 3


In [162]:
year, month, day = 2024, 3, 4
ticker_rank = filter_by_lastrank('earnings', year, month, day, 'zacks.com')

In [163]:
up_to_ranking = 2
for i in range(1,up_to_ranking + 1):
    print(f'{i}: ', ticker_rank[f'{i}'], '\n')

1:  ['TAST', 'TGS', 'HENKY'] 

2:  ['ALLK', 'HIBB', 'EOSE', 'EVA', 'IMMR', 'PASG', 'CMPX', 'SEMR', 'EKSO', 'AKYA', 'TIL', 'ASXC', 'VXRT', 'URG', 'TYRA'] 



In [164]:
ticker_rank = filter_by_lastrank('earnings', year, month, day, 'zacks.com')
initial_value_per_ticker = 1
total_initial_value = 0
total_final_value = 0
total_percentage = 0
start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 10))
end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 1))
for i in range(1,up_to_ranking + 1):
    for tic in ticker_rank[f'{i}']:
        yf_ticker = yf.download(tickers=tic, start=start_date, end =end_date, progress=False, ignore_tz=True)
        open_values = yf_ticker['Open'].values
        initial_value = open_values[-3]
        final_value = open_values[-1]
        
        percentage_difference = (open_values[-1] - open_values[-3])/open_values[-3]*100
        print(f'{tic}, {np.round(initial_value, 2)},   {np.round(final_value, 2)},  \
              {np.round(initial_value_per_ticker*percentage_difference,2)},    \
              {np.round(initial_value_per_ticker + (initial_value_per_ticker*percentage_difference/100),2)}')
        total_initial_value += initial_value_per_ticker
        total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

# compare with spy
yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

# compare with nasdaq
yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100


print(total_initial_value, np.round(total_final_value,2), \
      np.round((total_final_value - total_initial_value)/total_initial_value*100,2), \
      np.round(percentage_difference_spy,2), np.round(percentage_difference_ndaq,2))

TAST, 9.47,   9.49,                0.21,                  1.0
TGS, 13.2,   13.89,                5.23,                  1.05
HENKY, 17.1,   16.72,                -2.22,                  0.98
ALLK, 1.6,   1.62,                1.25,                  1.01
HIBB, 80.83,   82.0,                1.45,                  1.01
EOSE, 1.0,   1.05,                5.0,                  1.05
EVA, 0.42,   0.44,                4.76,                  1.05
IMMR, 6.8,   6.9,                1.47,                  1.01
PASG, 1.61,   1.72,                6.83,                  1.07
CMPX, 1.83,   1.92,                4.92,                  1.05
SEMR, 12.7,   12.94,                1.89,                  1.02
EKSO, 1.96,   2.0,                2.04,                  1.02
AKYA, 5.8,   6.13,                5.69,                  1.06
TIL, 11.92,   12.09,                1.43,                  1.01
ASXC, 0.31,   0.31,                0.0,                  1.0
VXRT, 1.29,   1.23,                -4.65,                  0

In [143]:
yf_ticker = yf.download(tickers='SPY', start='2024-03-01', end ='2024-03-06', progress=False, ignore_tz=True)


In [144]:
yf_ticker

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-03-01,508.980011,513.289978,508.559998,512.849976,512.849976,76805900
2024-03-04,512.030029,514.200012,512.000000,512.299988,512.299988,49799300
2024-03-05,510.239990,510.700012,504.910004,507.179993,507.179993,72670700


In [145]:
yf_ticker['Open'].values

array([508.98001099, 512.0300293 , 510.23999023])

(21, 91.68999999999998, 336.61904761904754, 70.68999999999998)